# Univariate Analysis

The objective of this notebook is to analyze each variable separately to understand its distribution, central tendencies (mean, median), and dispersion (variance, standard deviation). It also includes visualization of variables.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from src.utils import pie_plot, qq_plots, skewness_and_kurtosis, detect_outliers
import numpy as np




In [ ]:
data = pd.read_csv('../data/processed/df_fraud.csv')
data.head()

In [ ]:
data.info(show_counts=True)

# Visualization of the target variable

## isFraud

In [ ]:
data['isFraud'].describe()

In [ ]:
data['isFraud'].value_counts()

In [ ]:
data['isFraud'].value_counts(normalize=True).mul(100).apply(lambda x: f'{x:.3f}%')

Observations: 

The vast majority of transactions are non-fraudulent, which is evidenced by the very low mean (0.00129082) and the percentiles (25%, 50%, 75%) all being 0. The standard deviation is also low, indicating little variability. This is typical in fraud datasets, where fraudulent instances are much less common compared to non-fraudulent instances.

# Visualization of the categorical variables

## Type

In [ ]:
data['type'].describe(include = ['O'])

In [ ]:
data['type'].value_counts()

In [ ]:
pie_plot(data['type'])

Observations: 
- CASH_OUT & PAYMENT are the predominant types of transactions
- CASH_IN & TRANSFER are relatively popular
- DEBIT is the less common among types of transactions. 



## nameOrig and nameDest

In [ ]:

categorical_cols = [ 'nameOrig', 'nameDest']

fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(20, 10))

for ax, col in zip(axs, categorical_cols):
    top_categories = data[col].value_counts().nlargest(80)
    
    top_categories.plot(kind='bar', ax=ax)
    
    ax.set_title(f'Top 50 most frequent categories in {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Count')
    ax.grid(True)

plt.tight_layout()
plt.show()

Observations:
- nameOrig: There are many accounts that have sent money more than 1 time. Is there a relationship between fraudulent behaviours and the amount of times the account has sent money?

- nameDest: The amount of times that the most common accounts is way higher so for. Can I cluster them somehow? It gonna be interesting to analyse if in this case the accounts with high frequency have some relationship with the fraudulent behaviours too. 

# Visualization of the numerical variables

### Basic overview of the distributions for all variables

In [ ]:
numeric_cols =['step', 'amount','oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest','newbalanceDest']


fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(14,8))
axes = axes.flatten()  

for ax, col in zip(axes, numeric_cols):
    sns.kdeplot(data[col], ax=ax, fill=True)
    ax.set_title(f'Density Plot of {col}')
    ax.set_xlabel(col)
    ax.set_ylabel('Density')

plt.tight_layout()
plt.show()

Observations:

- The columns 'amount','oldbalanceOrig', 'newbalanceOrig', 'oldbalanceDest','newbalanceDest' have most of the values slighty above 0. 

- The column 'step' has  two ranges that look interesting and will be checked later more accurately.

In [ ]:
qq_plots(data, numeric_cols)

Observations:

- Positive skewness in every column. 

- As we said in the previous plots, step shows a bit more simetry but it remains far from being a normal distribution.

### Step

In [ ]:
data['step'].describe()

Observations:
- The mean (243.40) and median (239.00) are close, indicating relatively symmetric distribution. However, the standard deviation (142.33) suggests considerable variability in the data.
- The wide range (from 1 to 743) and the large difference between the 25th percentile (156) and the 75th percentile (335) suggest that there are high values or outliers in the dataset. This indicates potential right skewness (I'll check it later), with the distribution having a longer tail on the right side.

Since I have many rows, I'm going to use a density plot for better visualization 

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(data['step'], kde=True)
plt.title('Density plot of the variable "Step"')
plt.xlabel('Step')
plt.ylabel('Density')
plt.grid()
plt.show()

Observations:

For context, 1 step represents 1 hour. The total range of hour data is equivalent to a month. 

The are two main step ranges where the amount of transactions are very likely to happen. 
- Are they related to fraudulent behavior? 
- How is the behavior divided per day instead?

---

I'm going to check the distribution per day

In [ ]:
data['day_of_month'] = np.ceil(data['step'] / 24).astype(int)

In [ ]:
plt.figure(figsize=(10, 4))
sns.histplot(data['day_of_month'], kde=True)
plt.title('Histogram and Density of the variable "Day of Month"')
plt.xlabel('Day of month')
plt.ylabel('Density')
plt.grid()
plt.show()

Observations: 

The behavior is more clear with this graph. There are two main groups:

- Group 1: The ones that use the first and second day to do transactions. It makes sense since people get paid at the end of the month so they can use their money at the begining of the next month. 
- Group 2: The ones that take from 6 to 17 days to do transactions. That doesn't make sense to me. I'll try to understand this behavior later comparing it with the column isFraud and maybe other variables.. follow the path to the bivariate and multivariate analysis notebooks for more possible insights :)


In [ ]:
skewness_and_kurtosis(data, 'step')

Observations: 

- Skewness is very low, good, but the distribution is not normal. 
- Kurtosis is ver low (Platykurtic), light tails and flat peak so I won't find a huge amount of outliers

In [ ]:
skewness_and_kurtosis(data, 'day_of_month')

Observations:

The new approach based on days slightly increases both skewness and kurtosis

### Amount

In [ ]:
data['amount'].describe()

Observations:

- The mean (179,861.90) is significantly higher than the median (74,871.94), which, along with a high standard deviation (603,858.22), indicates that the distribution is heavily skewed to the right with a large number of high-value outliers.

- The minimum value is 0 and the maximum is 92,445,200, suggesting a wide range of values. The 25th percentile (13,389.57) and 75th percentile (208,721.50) show that the upper range of values contributes significantly to the high mean, further indicating right-skewness and potential outliers.

In [ ]:
count = data['amount'].value_counts()

plt.figure(figsize=(8, 4))
sns.histplot(count, kde=True, bins = 100)
plt.title('Density plot of the variable "Amount"')
plt.xlabel('Amount')
plt.ylabel('Density')
plt.grid()
plt.show()

I see almost nothing here, let's zoom in the small values of amount

In [ ]:
plt.figure(figsize=(10, 6))


sns.histplot(data[data['amount'] < 100000]['amount'], kde=True)
plt.title('Histogram of "Amount" (Values < 100000)')
plt.xlabel('Amount')
plt.ylabel('Density')

plt.tight_layout()
plt.grid()
plt.show()


In [ ]:
skewness_and_kurtosis(data, 'amount')

Observations:

Here we can see the distribution shape that reflect the basic statitstical data from before. Most transactions are below 40.000€. The distribution is very skewed and leptokurtic.

- Are this type of transactions related to fraudulent behaviour?

----

I'm going to use exponential increasing bins since the range of my data is high. Further analysis can give some insights using uniform bins.

In [ ]:
bins = [0, 1000, 10000, 100000, 1000000, 10000000, 100000000]
labels = ['0-1.000', '1.000-10.000', '10.000-100.000', '100.000-1.000.000', '1.000.000-10.000.000', '10.000.000-100.000.000']
data['amount_range'] = pd.cut(data['amount'], bins=bins, labels=labels, right=False)

In [ ]:
data['amount_range'].value_counts()

In [ ]:
data['amount_range'].describe()

In [ ]:
order = ['0-1.000', '1.000-10.000', '10.000-100.000', '100.000-1.000.000', '1.000.000-10.000.000', '10.000.000-100.000.000']


plt.figure(figsize=(8, 5))
barplot = sns.countplot(data=data, x='amount_range', order=order)

plt.title('Counts of Transactions per Amount Range')
plt.xlabel('Amount Range')
plt.ylabel('Count')

plt.xticks(rotation=45)  
plt.show()

Observations:

- The most commonn ranges are 10.000 - 100.000 and 100.000 - 1.000.000.
- The bins approach looks good to me even though other non exponential bin strategies could be considered. 
- I will study if this approach provides some insights about the relationship with fraudulent behaviours.

## oldbalanceOrig 

In [ ]:
data['oldbalanceOrig'].describe()

Observations:

- The mean of the dataset is 833,883.1, which is significantly higher than the median of 14,208.0. This discrepancy suggests that the data is right-skewed, meaning there are a few very high values pulling the mean upwards. The spread of the data is considerable, as indicated by the standard deviation of 2,888,243, which measures the average distance of each data point from the mean. This high standard deviation indicates that the data points are spread out over a large range of values. 
- The range of the data is from 0 to 59,585,040, further emphasizing the significant variability within the dataset. The interquartile range (IQR), which is the range of the middle 50% of the data, is 107,315.2. This IQR shows that the middle half of the data is also spread out, though not as extensively as the entire range. Overall, the distribution of oldbalanceOrig is right-skewed with a few extremely high values contributing to a high mean and substantial spread in the dataset.

In [ ]:
old_orig = data['oldbalanceOrig'].value_counts()
old_orig.head(10)

In [ ]:
filtered_old_orig = old_orig[old_orig >1]

In [ ]:
filtered_old_orig.head(90)

In [ ]:
integers = filtered_old_orig[filtered_old_orig.index % 1 == 0]
print(f'The length of the old balances with integers is: {len(integers)}')
integers.head(20)

In [ ]:
non_integers = filtered_old_orig[filtered_old_orig.index % 1 != 0]
print(f'The length of the old balances with decimals is: {len(non_integers)}')
non_integers.head(20)

In [ ]:
skewness_and_kurtosis(data,'newbalanceOrig')

Observations:

- There are repeated old balances over time. I have divided them into amounts of integers and amounts with decimals.

- There are many transactions with old balance = 0€. How is this possible? We should compare the type of transaction to check what could have been happening. 

- Repeated old balances with integers are much more frequent than old balances with decimals. Is it a coincidence? 

- High skewness and kurtosis

## newbalanceOrig

In [ ]:
data['newbalanceOrig'].describe()

Observations:

- The mean is quite high (855,113.7), but the median is 0. This suggests that while the average balance is relatively high, a large proportion of records have a balance of 0. This indicates a right-skewed distribution with a significant number of zero or near-zero balances.

- The standard deviation is very high (2,924,049), indicating a high level of variability in the data. This large dispersion suggests that there are some very high balances compared to the majority. The range of 49,585,040 indicates that the balances vary widely, from 0 to nearly 50 million, highlighting the presence of extreme values. The quartiles further confirm that a large portion of the data has low balances (including zero), with a smaller proportion having higher balances.

In [ ]:
new_orig = data['newbalanceOrig'].value_counts()
new_orig.head(10)

In [ ]:
filtered_new_orig = new_orig[new_orig >1]
filtered_new_orig.head(10)

In [ ]:
len(filtered_new_orig)

In [ ]:
integers = filtered_new_orig[filtered_new_orig.index % 1 == 0]
print(f'The length of the new balances with integers is: {len(integers)}')
integers.head()

In [ ]:
non_integers = filtered_new_orig[filtered_new_orig.index % 1 != 0]
print(f'The length of the new balances with decimals is: {len(non_integers)}')
non_integers.head(20)

In [ ]:
skewness_and_kurtosis(data,'newbalanceOrig')

Observations:

- High skewness and kurtosis

## oldbalanceDest

In [ ]:
data['oldbalanceDest'].describe()

- The mean value of 1,100,702 is significantly higher than the median value of 132,705.7. This suggests that the distribution is positively skewed, meaning that there are some extremely high values (outliers) pulling the mean up.


- The standard deviation is very high (3,399,180), indicating a large variability around the mean. This is consistent with the large difference between the mean and median, as well as the large range. The range of 356,015,900 shows a wide spread between the minimum and maximum values.The fact that both the 25th percentile and the median are much lower than the 75th percentile and maximum reinforces the idea of a right-skewed distribution, with a large proportion of low to mid-range values and a few very high values.

In [ ]:
old_dest = data['oldbalanceDest'].value_counts()
old_dest.head(20)

In [ ]:
filtered_old_dest = old_dest[old_dest >1]
filtered_old_dest.head()

In [ ]:
integers = filtered_old_dest[filtered_old_dest.index % 1 == 0]
print(f'The length of the old balances with integers is: {len(integers)}')
integers.head(20)

In [ ]:
non_integers = filtered_old_dest[filtered_old_dest.index % 1 != 0]
print(f'The length of the old balances with decimals is: {len(non_integers)}')
non_integers.head(20)

In [ ]:
skewness_and_kurtosis(data,'newbalanceOrig')

Observations:

- High skewness and kurtosis

## newbalanceDest

In [ ]:
data['newbalanceDest'].describe()

Observations:

- The mean and median values suggest a central tendency around a substantial amount. The mean (1,224,996) is higher than the median (214,661.4), indicating that while a substantial portion of the balances are around the median, the mean is significantly influenced by some very high balances. The large standard deviation (3,674,129) compared to the mean indicates high variability in the data. The spread of values is considerable, which is consistent with the maximum value being over 356 million.


- Both the 25th and 50th percentiles being at 0.0 and 214,661.4, respectively, highlight that while many entries are zero or low, there are also significant balances that push the mean higher. The 75th percentile value (1,111,909) shows that a substantial portion of the data has balances that are quite high compared to the median. The very high maximum value (356,179,300) indicates the presence of some extreme outliers. These extreme high values skew the mean upwards and contribute to the large standard deviation.

In [ ]:
new_dest = data['newbalanceDest'].value_counts()
new_dest.head(10)

In [ ]:
filtered_new_dest = new_dest[new_dest >1]

In [ ]:
integers = filtered_new_dest[filtered_new_dest.index % 1 == 0]
print(f'The length of the new balances with integers is: {len(integers)}')
integers.head(20)

In [ ]:
non_integers = filtered_new_dest[filtered_new_dest.index % 1 != 0]
print(f'The length of the new balances with decimals is: {len(non_integers)}')
non_integers.head(20)

In [ ]:
skewness_and_kurtosis(data,'newbalanceDest')

Observations:

It catches my attention the amount of exact same amounts of money moved. Maybe it's a coincidence but needs further analysis.

- Very high skewness and kurtosis.

### Balances in the destination

In [ ]:
subset_dest = data[['type', 'amount', 'oldbalanceDest', 'newbalanceDest', 'isFraud']].copy()
subset_dest['amount_moved'] = (subset_dest['newbalanceDest'] - subset_dest['oldbalanceDest']).round(2)
subset_dest = subset_dest.loc[subset_dest['amount'] != subset_dest['amount_moved']]
subset_dest

#### Potential losses in the destination

In [ ]:
subset_dest.query('isFraud == 1')

Observations:

Further investigation can be done here. Amounts don't match and there can be some patterns hidden related to user. Other features I would consider important are features related to location and transaction time

### Name Origin

In [ ]:
subset_orig_dest = data[['nameOrig', 'nameDest', 'isFraud']].copy()
subset_orig_dest

In [ ]:
# Group by nameOrig
nameorig_counts = subset_orig_dest.groupby('nameOrig').size()
nameorig_counts


In [ ]:
# Get the nameOrig with occurence > 1 
nameorig_common = nameorig_counts[nameorig_counts > 1]
subset_nameorig_repeated = data[data['nameOrig'].isin(nameorig_common.index)]

Can we find users in the origin with mixed fraudulent and not fraudulent behaviour? Are they only doing fraud? 

In [ ]:
fraudulent = subset_nameorig_repeated[subset_nameorig_repeated['isFraud'] == 1]['nameOrig'].unique()
non_fraudulent = subset_nameorig_repeated[subset_nameorig_repeated['isFraud'] == 0]['nameOrig'].unique()

common_fraud_nonfraud = np.intersect1d(fraudulent, non_fraudulent)
common_fraud_nonfraud

In [ ]:
fraud_nonfraud_in_orig = data[data['nameOrig'].isin(common_fraud_nonfraud)].sort_values(by = 'nameOrig', ascending = True).reset_index(drop = True)
fraud_nonfraud_in_orig.head()

Observations:

There is a very small amount of account data that shares fraudulent and non-fraudulent transactions. The only pattern observed is that every account has one fraudulent and one non-fraudulent transaction. Further analysis could be done here as long as we continue getting data to understand the behavior of these "hybrid" accounts

In [ ]:
two_times_nameorig = nameorig_counts[nameorig_counts == 2]
subset_data_two_times = data[data['nameOrig'].isin(two_times_nameorig.index)]
subset_two_data_filtered = subset_data_two_times[~subset_data_two_times['nameOrig'].isin(common_fraud_nonfraud)].sort_values(by = 'nameOrig', ascending = True).reset_index(drop = True)
subset_two_data_filtered


In [ ]:
subset_two_data_filtered.query('isFraud == 1')

In [ ]:
three_times_nameorig = nameorig_counts[nameorig_counts == 3]
subset_data_three_times = data[data['nameOrig'].isin(three_times_nameorig.index)].sort_values(by = 'nameOrig', ascending = True).reset_index(drop = True)
subset_data_three_times.head()

In [ ]:
subset_data_three_times.query('isFraud == 1')

Observations:

The remaining commonly used accounts (more than two times) only have non fraudulent transactions

### Name Destination

In [ ]:
# Group by nameOrig
namedest_counts = subset_orig_dest.groupby('nameOrig').size()
namedest_counts

In [ ]:
# Get the nameOrig with occurence > 1 
namedest_common = namedest_counts[namedest_counts > 1]
subset_namedest_repeated = data[data['nameDest'].isin(namedest_common.index)]

Can we find users in the destination with mixed fraudulent and not fraudulent behaviour? Are they only doing fraud? 

In [ ]:
fraudulent = subset_namedest_repeated[subset_namedest_repeated['isFraud'] == 1]['nameDest'].unique()
non_fraudulent = subset_namedest_repeated[subset_namedest_repeated['isFraud'] == 0]['nameDest'].unique()

common_fraud_nonfraud = np.intersect1d(fraudulent, non_fraudulent)
common_fraud_nonfraud

In [ ]:
two_times_namedest = namedest_counts[namedest_counts == 2]
subset_data_two_times = data[data['nameDest'].isin(two_times_namedest.index)]
subset_data_two_times.query('isFraud == 1')

In [ ]:
three_times_namedest = namedest_counts[namedest_counts == 3]
subset_data_three_times = data[data['nameDest'].isin(three_times_namedest.index)].sort_values(by = 'nameDest', ascending = True).reset_index(drop = True)
subset_data_three_times.head()

Observations:

Merchants only have 1 time fraud occurence. That can happen if the client rejects the transaction thinking it's an invalid but it is valid. Maybe the forgot they bought an item by the time the transaction was done.

## Outliers

In [ ]:
numeric_cols = data.select_dtypes(include=['int64','float64'])

plt.figure(figsize=(12, 8))
numeric_cols.boxplot()
plt.title('Boxplots of numerical variables')
plt.xticks(rotation=45)
plt.show()

In [ ]:
detect_outliers(data)